URL WORKING : https://www.google.com/travel/flights?hl=en&q=Flights%20to%20{GRU}%20from%20{BRU}%20on%20{2025-10-30}%20oneway
URL WORKING https://www.google.com/travel/flights?hl=en&q=Flights%20to%20{GRU}%20from%20{BRU}%20on%20{2025-10-30}%20oneway&curr=EUR
https://www.google.com/travel/flights?hl=en&q=Flights%20to%20{DESTI}%20from%20{ORI}%20on%20{YYYY-MM-DD}%20oneway&curr=CUR

--> je l'ai eu via https://github.com/celebi-pkg/flight-analysis/blob/main/src/google_flight_analysis/scrape.py

Il faut remplacer  DESTI ; ORI ; YYYY-MM-DD ; CUR



def create_url()

In [12]:
%pip install airportsdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.7/912.7 kB 24.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from airportsdata import load
airports= load('IATA')
ville ='Brussels'
resultats = airports.get('BRU')
resultats = {code: info for code, info in airports.items() if info['city'].lower() == ville.lower()}
print(resultats['BRU'])
print(resultats)

{'icao': 'EBBR', 'iata': 'BRU', 'name': 'Brussels Airport', 'city': 'Brussels', 'subd': 'Flanders', 'country': 'BE', 'elevation': 184.0, 'lat': 50.9014, 'lon': 4.48444, 'tz': 'Europe/Brussels', 'lid': ''}
{'BRU': {'icao': 'EBBR', 'iata': 'BRU', 'name': 'Brussels Airport', 'city': 'Brussels', 'subd': 'Flanders', 'country': 'BE', 'elevation': 184.0, 'lat': 50.9014, 'lon': 4.48444, 'tz': 'Europe/Brussels', 'lid': ''}, 'CRL': {'icao': 'EBCI', 'iata': 'CRL', 'name': 'Brussels South Charleroi Airport', 'city': 'Brussels', 'subd': 'Wallonia', 'country': 'BE', 'elevation': 614.0, 'lat': 50.4592, 'lon': 4.45382, 'tz': 'Europe/Brussels', 'lid': ''}}


In [2]:
import csv
import pandas as pd

csv_path = '/Users/focus_profond/GIT_repo/flight_price_tracker/Config/IATA_code.csv'
df = pd.read_csv(csv_path)
df.head()
result = df[(df['City'] == 'Brussels') & (df['Country'] == 'Belgium')]
result.info()

def get_IATA_CODE(city, country= None):
    """
    Get the IATA code for a given city and country.
    
    Parameters:
    - city (str): The name of the city.
    - country (str, optional): The name of the country. Defaults to None.
    
    Returns:
    - str: The IATA code if found, otherwise None.
    """
    result = df[(df['City'] == city)]# & (df['Country'] == country)]
    if result.empty:
        result = df[df['Airport name'].str.contains('Brussels', na=False)]

    if country is not None:
        result = df[(df['City'] == city) & (df['Country'] == country)]
        if result.empty:
            result = df[(df['City'] in city) & (df['Country'] == country)]
    else:
        result = df[df['City'] == city]
    
    if not result.empty:
        return result.iloc[0]['IATA']
    return None
iata = get_IATA_CODE('Brussels', 'Belgium')
#print(iata)

#--> pas super utile car Google Flights prend directmeent en compte les villes, pas besoin de passer par les IATA.

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IATA          0 non-null      object
 1   ICAO          0 non-null      object
 2   Airport name  0 non-null      object
 3   Country       0 non-null      object
 4   City          0 non-null      object
 5   Information   0 non-null      object
dtypes: object(6)
memory usage: 0.0+ bytes


TypeError: 'in <string>' requires string as left operand, not Series

In [9]:
def create_url(desti,ori, flight_date=None, currency='EUR'):
    """
    Create a Google Flights URL for a one-way flight search.
    
    Parameters:
    - desti: Destination airport code (e.g., 'GRU').
    - ori: Origin airport code (e.g., 'BRU').
    - flight_date: Date of the flight in 'YYYY-MM-DD' format (optional).
    - currency: Currency code (e.g., 'EUR') (optional).
    
    Returns:
    - str: Formatted Google Flights URL.
    """
    if flight_date:
        base_url = f"https://www.google.com/travel/flights?hl=en&q=Flights%20to%20{desti}%20from%20{ori}%20on%20{flight_date}%20oneway&curr={currency}"
    else:
        base_url = f"https://www.google.com/travel/flights?hl=en&q=Flights%20to%20{desti}%20from%20{ori}%20oneway&curr={currency}"
    #print(base_url)
    return base_url.format(desti=desti, ori=ori, flight_date=flight_date, currency=currency)

create_url('CDG','BRU')
create_url('CDG','BRU','2026-10-03')

'https://www.google.com/travel/flights?hl=en&q=Flights%20to%20CDG%20from%20BRU%20on%202026-10-03%20oneway&curr=EUR'

In [ ]:
config_dir = '/Users/focus_profond/GIT_repo/flight_price_tracker/Config'
excel_path = f'{config_dir}/trip_config_world.xlsx'
df = pd.read_excel(excel_path, sheet_name='good_one')
df['url'] = df.apply(lambda row: create_url(row['codeIATA_desti'],row['codeIATA_ori']) if pd.isna(row['url']) or row['url']=='' else row['url'], axis=1)
df.head(50)
df.to_csv(config_dir+'/trip_config_world.csv', index =False, encoding='utf-8')
excel_path_out = f'{config_dir}/trip_config_world_out.xlsx'
#df.to_excel(excel_path, sheet_name='good_one', index=False)
